In [1]:
import pandas as pd
import json
from pandas import json_normalize

## Creación de dataframes

#### Se requiere de 4 dataframes para obtener la información necesaria para cada Red Flag: Tender, Releases, Awards y Parties. Estos se formar a partir del archivo JSON generado anteriormente.

In [2]:
with open('../Data_input/response.json', 'r', encoding="utf8") as file:
        data = json.load(file)

### Releases

In [3]:
dict_releases = {'ocid':[], 'id':[], 'date':[], 'buyer_id':[], 'buyer_name':[], 'contracts_id':[],
                'contracts_status':[], 'contracts_awardID':[], 'contracts_value_amount':[],
                'contracts_value_currency':[], 'contracts_dateSigned':[], 'planning_budget_amount':[]}

In [4]:
for i in range(len(data)):
    dict_releases['ocid'].append(data[i]['releases'][0]['ocid'])
    dict_releases['id'].append(data[i]['releases'][0]['id'])
    dict_releases['date'].append(data[i]['releases'][0]['date'])
    dict_releases['buyer_id'].append(data[i]['releases'][0]['buyer']['id'])
    dict_releases['buyer_name'].append(data[i]['releases'][0]['buyer']['name'])
    try:
        dict_releases['contracts_id'].append(data[i]['releases'][0]['contracts'][0]['id'])
    except:
        dict_releases['contracts_id'].append(None)
    try:
        dict_releases['contracts_status'].append(data[i]['releases'][0]['contracts'][0]['status'])
    except:
        dict_releases['contracts_status'].append(None)
    try:
        dict_releases['contracts_awardID'].append(data[i]['releases'][0]['contracts'][0]['awardID'])
    except:
        dict_releases['contracts_awardID'].append(None)
    try:
        dict_releases['contracts_dateSigned'].append(data[i]['releases'][0]['contracts'][0]['dateSigned'])
    except:
        dict_releases['contracts_dateSigned'].append(None)
    try:
        dict_releases['contracts_value_amount'].append(data[i]['releases'][0]['contracts'][0]['value']['amount'])
    except:
        dict_releases['contracts_value_amount'].append(None)    
    try:
        dict_releases['contracts_value_currency'].append(data[i]['releases'][0]['contracts'][0]['value']['currency'])
    except:
        dict_releases['contracts_value_currency'].append(None)
    try:
        dict_releases['planning_budget_amount'].append(data[i]['releases'][0]['planning']['budget']['amount']['amount'])
    except:
        dict_releases['planning_budget_amount'].append(None)

In [5]:
releases = pd.DataFrame.from_dict(dict_releases)

In [6]:
releases.shape

(16128, 12)

In [7]:
releases.drop_duplicates(subset=["ocid"],keep="last", inplace=True)

In [8]:
releases.shape

(15417, 12)

In [9]:
releases['contracts_dateSigned'] = pd.to_datetime(releases['contracts_dateSigned']).dt.date
releases['contracts_dateSigned'] = pd.to_datetime(releases['contracts_dateSigned'])

In [10]:
releases.to_csv('../Data_input/releases.csv', index=False)

## Tender

In [11]:
dicc_tender = {'ocid':[],'id':[],'tenderPeriod':[],'procurementMethod':[],
              'value':[],'tenderers':[],'procuringEntity':[],
              'items':[],'numberOfTenderers':[],'enquiries':[],'enquiryPeriod':[],
               'mainProcurementCategory':[],'procurementMethodDetails':[]}

In [12]:
for i in range(len(data)):
    dicc_tender['ocid'].append(data[i]['releases'][0]['ocid'])
    for campo in dicc_tender.keys():
        if campo != 'ocid':
            try:
                dicc_tender[campo].append(data[i]['releases'][0]["tender"][campo])
            except:
                dicc_tender[campo].append(None)

In [13]:
df = pd.DataFrame(dicc_tender)

In [14]:
df.drop_duplicates(subset=["ocid"],keep="last", inplace=True)

In [15]:
df = df.reset_index(drop=True)

In [16]:
df.to_csv('../Data_input/tender.csv',index=False)

## Awards

In [17]:
# All
ocidDF = pd.DataFrame()
awardsDF = pd.DataFrame()
for d in data:
    ocid = d['releases'][0]['ocid']
    if (ocid != "ocds-5wno2w-003-MC-GPLR-2019-2805") and (ocid != "ocds-5wno2w-ARE-CBSE-001-2019-222424") and (ocid != "ocds-5wno2w-CB-FAME-001-2019-510646") and (ocid != "ocds-5wno2w-CBS-EPMAAM-001-2019-699688") and (ocid != "ocds-5wno2w-CDC-GADPRP-2019-001-107859") and (ocid != "ocds-5wno2w-CDC-GADPSO-2019-004-145156") and (ocid != "ocds-5wno2w-003-MC-GPLR-2019-2805") :
        awards = d['releases'][0]['awards']
        df = pd.DataFrame([{"ocid":ocid}])
        df2 = pd.DataFrame(awards)
        ocidDF = pd.concat([ocidDF,df], ignore_index=True)
        awardsDF = pd.concat([awardsDF,df2], ignore_index=True)

In [18]:
awardsDF = pd.concat([ocidDF,awardsDF], axis=1)

In [19]:
awardsDF.reset_index(drop=True, inplace=True)
awardsDF.rename(columns = {'id':'id_award'}, inplace = True)

In [20]:
awardsDF.drop_duplicates(subset=["ocid"],keep="last", inplace=True)

In [21]:
awardsDF = awardsDF.reset_index(drop=True)

In [22]:
awardsDF.shape

(15417, 9)

In [23]:
awardsDF.to_csv('../Data_input/awards.csv', index=False)

## Parties

In [24]:
def comprobarExistencia(dicc,llave):
    try:
        a=dicc[llave]
        return a
    except:
        return None

In [25]:
diccParties={"ocid":[],"id":[],"name":[],"roles":[],"address_region":[],"address_locality":[],"address_countryName":[],
            "address_streetAddress":[],"address_postalCode":[],"identifier_id":[],"identifier_scheme":[],
            "identifier_legalName":[],"contactPoint_name":[],"contactPoint_url":[],"contactPoint_email":[],
            "contactPoint_telephone":[]}

In [26]:
for i in data:
    caso=i['releases'][0]
    ocid=caso["ocid"]
    parties=caso['parties']
    diccParties["ocid"].append(ocid)
    lista_id=[]
    lista_name=[]
    lista_roles=[]
    lista_address_region=[]
    lista_address_locality=[]
    lista_address_countryName=[]
    lista_address_streetAddress=[]
    lista_address_postalCode=[]
    lista_identifier_id=[]
    lista_identifier_scheme=[]
    lista_identifier_legalName=[]
    lista_contactPoint_name=[]
    lista_contactPoint_url=[]
    lista_contactPoint_email=[]
    lista_contactPoint_telephone=[]
    for tender in parties:
        lista_id.append(comprobarExistencia(tender,"id"))
        lista_name.append(comprobarExistencia(tender,"name"))
        lista_roles.append(comprobarExistencia(tender,"roles"))
        lista_address_region.append(comprobarExistencia(comprobarExistencia(tender,"address"),"region"))
        lista_address_locality.append(comprobarExistencia(comprobarExistencia(tender,"address"),"locality"))
        lista_address_countryName.append(comprobarExistencia(comprobarExistencia(tender,"address"),"countryName"))
        lista_address_streetAddress.append(comprobarExistencia(comprobarExistencia(tender,"address"),"streetAddress"))
        lista_address_postalCode.append(comprobarExistencia(comprobarExistencia(tender,"address"),"postalCode"))
        lista_identifier_id.append(comprobarExistencia(comprobarExistencia(tender,"identifier"),"id"))
        lista_identifier_scheme.append(comprobarExistencia(comprobarExistencia(tender,"identifier"),"scheme"))
        lista_identifier_legalName.append(comprobarExistencia(comprobarExistencia(tender,"identifier"),"legalName"))
        lista_contactPoint_name.append(comprobarExistencia(comprobarExistencia(tender,"contactPoint"),"name"))
        lista_contactPoint_url.append(comprobarExistencia(comprobarExistencia(tender,"contactPoint"),"url"))
        lista_contactPoint_email.append(comprobarExistencia(comprobarExistencia(tender,"contactPoint"),"email"))
        lista_contactPoint_telephone.append(comprobarExistencia(comprobarExistencia(tender,"contactPoint"),"telephone"))
    diccParties["id"].append(lista_id)
    diccParties["name"].append(lista_name)
    diccParties["roles"].append(lista_roles)
    diccParties["address_region"].append(lista_address_region)
    diccParties["address_locality"].append(lista_address_locality)
    diccParties["address_countryName"].append(lista_address_countryName)
    diccParties["address_streetAddress"].append(lista_address_streetAddress)
    diccParties["address_postalCode"].append(lista_address_postalCode)
    diccParties["identifier_id"].append(lista_identifier_id)
    diccParties["identifier_scheme"].append(lista_identifier_scheme)
    diccParties["identifier_legalName"].append(lista_identifier_legalName)
    diccParties["contactPoint_name"].append(lista_contactPoint_name)
    diccParties["contactPoint_url"].append(lista_contactPoint_url)
    diccParties["contactPoint_email"].append(lista_contactPoint_email)
    diccParties["contactPoint_telephone"].append(lista_contactPoint_telephone)

In [27]:
df=pd.DataFrame(diccParties)

In [28]:
df.drop_duplicates(subset=["ocid"], keep="last", inplace = True)

In [29]:
df=df.reset_index(drop=True)

In [30]:
parties=df
parties = parties[parties['ocid'] != 'ocds-5wno2w-003-MC-GPLR-2019-2805']
parties = parties[parties['ocid'] != 'ocds-5wno2w-ARE-CBSE-001-2019-222424']
parties = parties[parties['ocid'] != 'ocds-5wno2w-CB-FAME-001-2019-510646']
parties = parties[parties['ocid'] != 'ocds-5wno2w-CBS-EPMAAM-001-2019-699688']
parties.reset_index(drop=True, inplace=True)

In [31]:
parties.shape

(15417, 16)

In [32]:
parties.to_csv("../Data_input/parties.csv")